# Temporal Factor Calculator
## Calculate monthly factor and export result as csv

In [ ]:
from utility import db_connect, query2csv
from settings import  DBNAME, DBPASS, DBUSER, DBHOST

qsql="""
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
V_jmyl as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      baa_ex_sus.analysis_areas_daily_volume as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
  ),
madt as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from
      v_jmyl
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
aadt as (
select
  analysis_area_id,
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd 
    on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
monthly_madt_exclude_holiday as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume) as madt
  from
      V_jmyl_exclude_holiday
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
fm as (
select
  madt_nh.analysis_area_id,
  madt_nh.month,
  madt_nh.year,
  round(madt_nh.madt/aadt.aadt::numeric,2) as monthly_factor
from
  monthly_madt_exclude_holiday as madt_nh inner join aadt 
    using(analysis_area_id, year)
where
  aadt.aadt <> 0  
)
select * from fm
order by 1,3,2
"""
csvfile='monthly_factor.csv'
query2csv(qsql,csvfile)

## Monthly factor group by city and mode

In [ ]:
qsql="""
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
V_jmyl as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      baa_ex_sus.analysis_areas_daily_volume as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
  ),
madt as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from
      v_jmyl
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
aadt as (
select
  analysis_area_id,
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd 
    on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
monthly_madt_exclude_holiday as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume) as madt
  from
      V_jmyl_exclude_holiday
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
fm as (
select
  madt_nh.analysis_area_id,
  madt_nh.month,
  madt_nh.year,
  round(madt_nh.madt/aadt.aadt::numeric,2) as fm
from
  monthly_madt_exclude_holiday as madt_nh 
    inner join aadt using(analysis_area_id, year)
where
  aadt.aadt <> 0  
),
city_mode_group as (
select 
  ar.analysis_area_name as city,
  aa.mode,
  array_agg(analysis_area_id order by analysis_area_id) 
    as analysis_area_id_list
from
baa.analysis_area_regions as ar,
baa.analysis_areas as aa
where ar.analysis_area_regions_id = aa.analysis_area_regions_id
group by 1,2
)  
select
  cmg.city,
  cmg.mode,
  cmg.analysis_area_id_list,
  fm.month,
  fm.year,
  round(avg(fm.fm), 2) as fm_cmp
 
from
fm inner join city_mode_group as cmg
on fm.analysis_area_id = Any(cmg.analysis_area_id_list::int[])
group by 1,2,3,5,4
order by 1,2,3,5,4
"""
csvfile='monthly_factor_group.csv'
query2csv(qsql,csvfile)

#### Calculate monthly factor by analysis_area_id

In [ ]:
from utility import db_connect, query2csv
from settings import  DBNAME, DBPASS, DBUSER, DBHOST

def monthly_factor_by_site(analysis_area_id):
    query = """
with d as (
          select generate_series(0,6) as dayofweek
        ),
        m as (
          select generate_series(1,12) as month
        ),
        hrly as (
        select
          baaa.analysis_area_id,
          date_trunc('day', bpd.start_time) as date,
          to_char(bpd.start_time, 'HH24') as hour,
          sum(bpd.volume) as volume
        from
          baa.analysis_areas as baaa 
            inner join baa_ex_sus.data as bpd 
            on bpd.flow_detector_id = Any(baaa.flow_detector_list::int[])
        where 
          baaa.analysis_area_id = {0}     
          group by analysis_area_id, bpd.start_time
        ),
        daily as (
        select 
          analysis_area_id,
          date_trunc('day', date) as date,
          sum(volume) as volume
        from 
        hrly
        group by analysis_area_id, date_trunc('day', date) 
        ) ,
V_jmyl as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
  ),
madt as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from
      v_jmyl
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
aadt as (
select
  analysis_area_id,
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  daily as baaad
  left join baa.holidays as baahd 
    on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
monthly_madt_exclude_holiday as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume) as madt
  from
      V_jmyl_exclude_holiday
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
fm as (
select
  madt_nh.analysis_area_id,
  madt_nh.month,
  madt_nh.year,
  round(madt_nh.madt/aadt.aadt::numeric,2) as monthly_factor
from
  monthly_madt_exclude_holiday as madt_nh inner join aadt 
    using(analysis_area_id, year)
where
  aadt.aadt <> 0  
)
select * from fm
order by 1,3,2 
    """.format(analysis_area_id)
    conn = db_connect()
    with conn:
        with conn.cursor() as curs:
            curs.execute(query)
            rows = curs.fetchall()
            return rows

In [ ]:
analysis_area_id = 233
monthly_factor = monthly_factor_by_site(analysis_area_id)
print ('monthly factor for analysis area:{0} {1}'.format( 
              analysis_area_id, monthly_factor))

### Calculate Hour factor and export result in csv

In [ ]:
from utility import db_connect, query2csv
from settings import  DBNAME, DBPASS, DBUSER, DBHOST

qsql="""
with v_h_7_8 as (
select 
  baadv.analysis_area_id,
  avg(baadv.volume) volume
from 
  baa_ex_sus.analysis_areas_hourly_volume as baadv
where
  baadv.hour in ('07', '08')
  and extract(dow from baadv.date) in (1,2,3,4,5)
  group by baadv.analysis_area_id
  ),
  v_h_11_12 as (
select 
  baadv.analysis_area_id,
  avg(baadv.volume) volume
from 
  baa_ex_sus.analysis_areas_hourly_volume as baadv
where
  baadv.hour in ('11', '12')
  and extract(dow from baadv.date) in (1,2,3,4,5)
  group by baadv.analysis_area_id
  ),
  ami as (
  select
    vh78.analysis_area_id,
    round(vh78.volume, 2) as vh_78,
    round(vh1112.volume,2) as vh_11_12,
    round(vh78.volume/vh1112.volume, 2) as ami,
    case 
     when (round(vh78.volume/vh1112.volume, 2) <= 0.7) 
       then 'Hourly Noon Activity'
     when (round(vh78.volume/vh1112.volume, 2) >  1.4) 
       then 'Hourly Commute'
     ELSE 'Hourly Multipurpose'
  END as hour_group  
  from 
    v_h_7_8 as vh78 inner join v_h_11_12 as vh1112 
      using(analysis_area_id)
  ),
  ami_fg as (
  select 
    ami.analysis_area_id,
  baaa.mode,
  baaa.analysis_area_name,
  ami.vh_78,
  ami.vh_11_12,
  ami.ami,
  ami.hour_group as hourly_group
from
   ami inner join baa.analysis_areas as baaa using(analysis_area_id)  
   )
select * from ami_fg 
"""
csvfile='hourly_factor_ami.csv'
query2csv(qsql,csvfile)

### Calculate 84 factors and export csv with factors

In [ ]:
qsql = """
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
-- v_ijmy:Compute an average by day of week for each month.
v_ijmy as (
  select 
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      baa_ex_sus.analysis_areas_daily_volume as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
-- madt: average volume each month, each year for sites
madt as (  
  select 
      analysis_area_id,
      month,
      year,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from 
      v_ijmy
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
AADT as (
select 
  analysis_area_id, 
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd 
    on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
-- 84 factors volume count should exclude holiday weeks
factor84 as (
select 
  v_jmyl_nh.analysis_area_id,
  v_jmyl_nh.volume as v_jmyl,
  AADT.aadt as aadt,
  round(v_jmyl_nh.volume/aadt::numeric, 2) as f_jmys,
  v_jmyl_nh.dayofweek,
  v_jmyl_nh.month,
  v_jmyl_nh.year
from
  V_jmyl_exclude_holiday as v_jmyl_nh inner join AADT 
    using(analysis_area_id, year)
where
  AADT.AADT <> 0
)
select * from factor84
order by 1,7,6,5
"""
csvfile='84_factor.csv'
query2csv(qsql,csvfile)

### 84 factor group by city, mode and WWI type

In [ ]:
qsql="""
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
-- v_ijmy:Compute an average by day of week for each month.
v_ijmy as (
  select 
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      baa_ex_sus.analysis_areas_daily_volume as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
-- madt: average volume each month, each year for sites
madt as (  
  select 
      analysis_area_id,
      month,
      year,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from 
      v_ijmy
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
AADT as (
select 
  analysis_area_id, 
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
--daily_exclude_holiday:daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd 
    on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
-- 84 factors volume count should exclude holiday weeks
factor84 as (
select 
  v_jmyl_nh.analysis_area_id,
  v_jmyl_nh.volume as v_jmyl,
  AADT.aadt as aadt,
  round(v_jmyl_nh.volume/aadt::numeric, 2) as f_jmys,
  v_jmyl_nh.dayofweek,
  v_jmyl_nh.month,
  v_jmyl_nh.year
from
  V_jmyl_exclude_holiday as v_jmyl_nh 
    inner join AADT using(analysis_area_id, year)
where
  AADT.AADT <> 0
),
V_we as (
select 
  baadv.analysis_area_id,
  avg(baadv.volume) vwe
from 
  baa_ex_sus.analysis_areas_daily_volume as baadv
where 
  extract(dow from baadv.date) in (0,6)
  group by baadv.analysis_area_id
),
V_wd as (
select 
  baadv.analysis_area_id,
  avg(baadv.volume) vwd
from 
  baa_ex_sus.analysis_areas_daily_volume as baadv
where 
  extract(dow from baadv.date) in (1,2,3,4,5)
  group by baadv.analysis_area_id
),
grouping as (
select 
  V_we.analysis_area_id,
  round(V_we.vwe, 2) as V_we,
  round(V_wd.vwd, 2) as V_wd,
  round(V_we.vwe/V_wd.vwd, 2) as wwi,
  case 
     when (round(V_we.vwe/V_wd.vwd, 2) <= 0.8) 
       then 'Weekday Commute'
     when (round(V_we.vwe/V_wd.vwd, 2) >  1.2) 
       then 'Weekend Multipurpose'
     ELSE 'Weekly Multipurpose'
  END as grouping   
from 
  V_we inner join V_wd using (analysis_area_id)
),
wwi as (
select
  grouping.analysis_area_id,
  baaa.mode,
  baaa.analysis_area_name,
  baaa.analysis_area_regions_id,
  grouping.v_we,
  grouping.v_wd,
  grouping.wwi,
  grouping.grouping as weekly_group
from
   grouping inner join baa.analysis_areas 
     as baaa using(analysis_area_id)  
),
factorgrp as (
select 
  ar.analysis_area_name as city, 
  wwi.mode,
  wwi.weekly_group,
  array_agg(wwi.analysis_area_id order by analysis_area_id) 
    as analysis_area_id_list
from 
  wwi, baa.analysis_area_regions as ar
where 
  ar.analysis_area_regions_id = wwi.analysis_area_regions_id
  group by 1,2,3
)
  select
    fg.city,
    fg.weekly_group,
    fg.mode,
    fg.analysis_area_id_list,
    f84.dayofweek, 
    f84.month,
    f84.year,
    round(avg(f84.f_jmys), 2) as f_jmys_avg
  from 
    factor84 as f84 inner join factorgrp as fg
    on f84.analysis_area_id = Any(fg.analysis_area_id_list::int[])
    group by     
    fg.city,
    fg.weekly_group,
    fg.mode,
    fg.analysis_area_id_list,
    f84.dayofweek, 
    f84.month,
    f84.year
    order by fg.city,
    fg.weekly_group,
    fg.mode,f84.year, f84.month, f84.dayofweek
"""
csvfile='84factor_groups.csv'
query2csv(qsql,csvfile)   

#### Calculate 84 factor by analysis_area_id

In [ ]:
from utility import db_connect, query2csv
from settings import  DBNAME, DBPASS, DBUSER, DBHOST

def factor84_by_site(analysis_area_id):
    query = """
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
hrly as (
  select
    baaa.analysis_area_id,
    date_trunc('day', bpd.start_time) as date,
    to_char(bpd.start_time, 'HH24') as hour,
    sum(bpd.volume) as volume
  from
    baa.analysis_areas as baaa 
    inner join baa_ex_sus.data as bpd 
      on bpd.flow_detector_id = Any(baaa.flow_detector_list::int[])
  where 
     baaa.analysis_area_id = {0}    
     group by analysis_area_id, bpd.start_time
),
daily as (
   select 
     analysis_area_id,
     date_trunc('day', date) as date,
     sum(volume) as volume
   from 
     hrly
     group by analysis_area_id, date_trunc('day', date) 
),
-- v_ijmy:Compute an average by day of week for each month.
v_ijmy as (
  select 
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
-- madt: average volume each month, each year for sites
madt as (  
  select 
      analysis_area_id,
      month,
      year,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from 
      v_ijmy
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
AADT as (
select 
  analysis_area_id, 
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  daily as baaad
  left join baa.holidays as baahd 
    on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
-- 84 factors volume count should exclude holiday weeks
factor84 as (
select 
  v_jmyl_nh.analysis_area_id,
  v_jmyl_nh.volume as v_jmyl,
  AADT.aadt as aadt,
  round(v_jmyl_nh.volume/aadt::numeric, 2) as f_jmys,
  v_jmyl_nh.dayofweek,
  v_jmyl_nh.month,
  v_jmyl_nh.year
from
  V_jmyl_exclude_holiday as v_jmyl_nh inner join AADT 
    using(analysis_area_id, year)
where
  AADT.AADT <> 0
)
select * from factor84
order by 1,7,6,5
    """.format(analysis_area_id)
    conn = db_connect()
    with conn:
        with conn.cursor() as curs:
            curs.execute(query)
            rows = curs.fetchall()
            return rows

In [ ]:
analysis_area_id = 233
f84_by_site = factor84_by_site(analysis_area_id)
print ('monthly factor for analysis area:{0} {1}'.format( 
          analysis_area_id, f84_by_site))